In [1]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt
import matplotlib

matplotlib.style.use('ggplot')

In [2]:
df = pd.read_csv("sanfrancisco_incidents_summer_2014.csv")

In [3]:
df.groupby('Category').size().plot(kind='bar')
plt.show()